In [4]:
import os
import pandas as pd
from PIL import Image

In [5]:
classes = {
    'Banh Beo': 'bánh bèo',
    'Banh Can': 'bánh căn',
    'Banh Gio': 'bánh giò',
    'Banh Mi': 'bánh mì',
    'Banh Trang Nuong': 'bánh tráng nướng',
    'Banh Xeo': 'bánh xèo',
    'Bap Xao': 'bắp xào',
    'Bun Bo': 'bún bò',
    'Bun Cha': 'bún chả',
    'Bun Dau': 'bún đậu',
    'Bun Mam': 'bún mắm',
    'Bun Thit Nuong': 'bún thịt nướng',
    'Cao Lau': 'cao lầu',
    'Chao Long': 'cháo lòng',
    'Com Tam': 'cơm tấm',
    'Goi Cuon': 'gỏi cuốn',
    'Hu Tieu': 'hủ tiếu',
    'Mi Quang': 'mì quảng',
    'Pha Lau': 'phá lấu',
    'Pho': 'phở',
}

In [6]:
train = pd.read_csv('annotations/raw/train.csv')   
test = pd.read_csv('annotations/raw/test.csv')
validation = pd.read_csv('annotations/raw/validation.csv')

In [7]:
train.describe()

,question,answer,question_type,answer_type,image_path
count,41639,41639,41639,41639,41639
unique,3412,54,3,2,6867
top,Có nước chấm đi kèm không?,Có,yes/no,text,train/Banh_Beo/10.jpg
freq,1510,16154,17298,23482,14


In [8]:
test['question_type'].unique(), test['answer_type'].unique()

(array(['recognition', 'yes/no', 'color', 'regconition'], dtype=object),
 array(['text', 'boolean'], dtype=object))

In [9]:
test = test[test['question_type'].isin(['recognition', 'yes/no', 'color'])]
test = test[test['answer_type'].isin(['text', 'boolean'])]
validation = validation[validation['question_type'].isin(['recognition', 'yes/no', 'color'])]
validation = validation[validation['answer_type'].isin(['text', 'boolean'])]

In [10]:
test.describe()

,question,answer,question_type,answer_type,image_path
count,10533,10533,10533,10533,10533
unique,1375,44,3,2,1870
top,Màu sắc chính của món ăn là gì?,Có,yes/no,text,test/Pho/26.jpg
freq,784,3776,4306,6115,9


In [11]:
validation.describe()

,question,answer,question_type,answer_type,image_path
count,5522,5522,5522,5522,5522
unique,948,42,3,2,983
top,Màu sắc chính của món ăn là gì?,Có,yes/no,text,validation/Banh_Beo/14.jpg
freq,373,1930,2243,3221,9


In [12]:
train.head()

,question,answer,question_type,answer_type,image_path
0,Món này là món gì?,Banh Beo,recognition,text,train/Banh_Beo/1.jpg
1,Đây có phải là Banh Beo không?,Có,yes/no,boolean,train/Banh_Beo/1.jpg
2,Màu chủ đạo của Banh Beo là gì?,Trắng,color,text,train/Banh_Beo/1.jpg
3,Banh Beo có màu gì?,Trắng,color,text,train/Banh_Beo/1.jpg
4,Banh Beo có nước chấm không?,Có,yes/no,boolean,train/Banh_Beo/1.jpg


In [13]:
def fix_dishes_name(data, classes):
    for k, v in classes.items():
        data['question'] = data.question.str.replace(k, v)
        data['answer'] = data.answer.str.replace(k, v)        
    return data

In [14]:
def normalize_text(data):
    data['question'] = data.question.str.lower()
    data['question_type'] = data.question_type.str.lower()
    data['answer'] = data.answer.str.lower()
    data['answer_type'] = data.answer_type.str.lower()
    
    data['question'] = data.question.str.replace('?', '')
    
    return data
    

In [15]:
def remove_unpopular_answer(data):
    answers = {}
    for answer in data.answer:
        if answer not in answers.keys():
            answers[answer] = 1
        else:
            answers[answer] += 1
            
    answers = sorted(answers.items(), key=lambda x: x[1], reverse=True)
    unpopular_answers = [key for key, value in answers if value < 20]
    
    return data[~data.answer.isin(unpopular_answers)]
    


In [16]:
# def clean_type(data):
#     question_types = ['recognition', 'yes/no', 'color']
#     answer_types = ['text', 'boolean']
    
#     data = data[~data['question_type'].isin(question_types)]
#     data = data[~data['answer_type'].isin(answer_types)]
    
#     return data

In [17]:
def preprocess(data, classes):
    data = fix_dishes_name(data, classes)
    data = normalize_text(data)
    data = remove_unpopular_answer(data)
    # data = clean_type(data)
    return data

In [18]:
train = preprocess(train, classes)
test = preprocess(test, classes)
validation = preprocess(validation, classes)

In [19]:
train.describe()

,question,answer,question_type,answer_type,image_path
count,41613,41613,41613,41613,41613
unique,2757,27,3,2,6867
top,có nước chấm đi kèm không,có,yes/no,text,train/Banh_Beo/10.jpg
freq,1510,16154,17289,23467,14


In [20]:
test.describe()

,question,answer,question_type,answer_type,image_path
count,10530,10530,10530,10530,10530
unique,1154,27,3,2,1870
top,màu sắc chính của món ăn là gì,có,yes/no,text,test/Pho/26.jpg
freq,784,3776,4306,6112,9


In [21]:
validation.describe()

,question,answer,question_type,answer_type,image_path
count,5518,5518,5518,5518,5518
unique,805,27,3,2,983
top,màu sắc chính của món ăn là gì,có,yes/no,text,validation/Banh_Beo/14.jpg
freq,373,1930,2242,3218,9


In [22]:
train.answer.unique(), test.answer.unique(), validation.answer.unique()

(array(['bánh bèo', 'có', 'trắng', 'vàng', 'đỏ', 'đúng', 'không', 'nâu',
        'bánh căn', 'bánh giò', 'bánh mì', 'bánh tráng nướng', 'bánh xèo',
        'bắp xào', 'bún bò', 'bún chả', 'bún đậu', 'bún mắm',
        'bún thịt nướng', 'cao lầu', 'cháo lòng', 'cơm tấm', 'gỏi cuốn',
        'hủ tiếu', 'mì quảng', 'phở', 'phá lấu'], dtype=object),
 array(['bánh bèo', 'có', 'trắng', 'đỏ', 'nâu', 'không', 'vàng',
        'bánh căn', 'bánh giò', 'đúng', 'bánh mì', 'bánh tráng nướng',
        'bánh xèo', 'bắp xào', 'bún bò', 'bún chả', 'bún đậu', 'bún mắm',
        'bún thịt nướng', 'cao lầu', 'cháo lòng', 'cơm tấm', 'gỏi cuốn',
        'hủ tiếu', 'mì quảng', 'phá lấu', 'phở'], dtype=object),
 array(['bánh bèo', 'có', 'trắng', 'không', 'vàng', 'đỏ', 'đúng',
        'bánh căn', 'nâu', 'bánh giò', 'bánh mì', 'bánh tráng nướng',
        'bánh xèo', 'bắp xào', 'bún bò', 'bún chả', 'bún đậu', 'bún mắm',
        'bún thịt nướng', 'cao lầu', 'cháo lòng', 'cơm tấm', 'gỏi cuốn',
        'hủ tiếu', 'm

In [23]:
answers_in_test_not_in_train = set(train['answer']) - set(test['answer'])
print(answers_in_test_not_in_train)

set()


In [24]:
answers_in_test_not_in_train = set(train['answer']) - set(validation['answer'])
print(answers_in_test_not_in_train)

set()


In [25]:
test.to_csv('annotations/test.csv', index=False)
validation.to_csv('annotations/validation.csv', index=False)